In [1]:
from pathlib import Path
from scipy.io import loadmat

import pandas as pd


from reciprocal_isomap import ReciprocalIsomap

import numpy as np 
from kmapper import KeplerMapper
from landmark_cover import LandmarkCover
from sklearn.cluster import AgglomerativeClustering

/home/poetz/anaconda3/envs/tda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load one patient data as X = time x ROI


base_dir = Path(os.getcwd()).parent 
data_dir = base_dir / 'data'
data = loadmat(data_dir / 'timeseries_allsubs.mat')
# subs, timepoints, regions
data['timeseries'].shape

X = data['timeseries'][0]

X.shape

#Generate encoding matrix of timing labels

labels = loadmat(data_dir / 'labels.mat')
labels = [e[0] for e in labels['labels'].squeeze()]

df = pd.DataFrame(labels, columns=['labels'])
target, labels = pd.factorize(df.labels.values)
y = pd.DataFrame({l:(target==i).astype(int) for i,l in enumerate(labels)})

In [10]:
# compute distances with reciprocal isomap


# compute geodesic distances on a reciprocal neighbor graph
r_isomap = ReciprocalIsomap(n_neighbors=30,metric='correlation',neighbors_mode="connectivity", n_jobs=64)
geodesic_distances = r_isomap._reciprocal_distances(X).toarray()


In [11]:
geodesic_distances

array([[0., 1., 1., ..., 7., 7., 4.],
       [1., 0., 1., ..., 6., 6., 3.],
       [1., 1., 0., ..., 6., 6., 3.],
       ...,
       [7., 6., 6., ..., 0., 1., 7.],
       [7., 6., 6., ..., 1., 0., 7.],
       [4., 3., 3., ..., 7., 7., 0.]])

In [12]:
#feed distances to KeplerMapper using Landmark Cover



# setup KeplerMapper object
mapper = KeplerMapper(verbose=1)

# run KeplerMapper using the LandmarkCover
graph = mapper.map(
    lens=geodesic_distances, X=X,
    cover=LandmarkCover(n_landmarks=400, perc_overlap=0.35, metric='precomputed'),
    clusterer=AgglomerativeClustering(n_clusters=10, linkage='single'),
    remove_duplicate_nodes=True,
)

# visualize the graph
html = mapper.visualize(graph, path_html=f'prova0.html')

KeplerMapper()
Mapping on data shaped (4340, 100) using lens shaped (4340, 4340)

Creating 401 hypercubes.
Merged 1965 duplicate nodes.

Number of nodes before merger: 4010; after merger: 2045


Created 261540 edges and 2045 nodes in 0:00:50.070079.
Wrote visualization to: prova0.html


In [9]:
## Visualize the shape graph using DyNeuSR's DyNeuGraph
from dyneusr import DyNeuGraph

dG = DyNeuGraph(G=graph, y=y)
dG.visualize('trial_0.html')

label,group,value,row_count
REST,0,2400,4340
EMOTION,1,176,4340
GAMBLING,2,253,4340
LANGUAGE,3,316,4340
MOTOR,4,284,4340
RELATIONAL,5,232,4340
SOCIAL,6,274,4340
MEMORY,7,405,4340


Already serving localhost:None 
[Force Graph] file:///home/poetz/phd_projects/CSSS22-TDA/code/trial_0.html


DyNeuGraph()